#### Solver

The solver workflow begins by sending the language model a detailed instruction to output a JSON object with exactly two sections—“variables” and “constraints”—where each variable entry names a symbol and lists its allowed values, and each constraint entry specifies which variables participate and provides a Python‐style Boolean expression (as a string) that enforces the logical relationship. Once the model returns that JSON, it is parsed into a native data structure, and the solver first checks that every variable mentioned in any constraint actually appears in the variables list, raising a clear error if any name is undefined. Next, each declared variable and its domain are registered with the constraint‐solving engine so it knows which symbols exist and what values they may take. Finally, each constraint’s expression string is converted into a callable function and associated with its variable list, instructing the engine to enforce that relationship during search. At the end of this process, the solver has a complete constraint network—variables with domains and constraints as functions—and invoking the solver yields all assignments that satisfy every condition, or else it reports a descriptive error if any step (undefined variable or invalid expression) fails.

In [13]:
import os
from utils.AzureAdapter import AzureAdapter
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("AZURE_API_KEY")
api_endpoint = os.getenv("AZURE_API_ENDPOINT")
api_version = os.getenv("AZURE_API_VERSION")
deployment_name = "gpt-4o"

llm = AzureAdapter(api_key=api_key, api_endpoint=api_endpoint, api_version=api_version)

#### Solver

In [71]:
solver_system_prompt = """
You are an expert logic puzzle parser. Your task is to extract from a given natural language puzzle description all variables and constraints, and output a JSON object that can later be used to construct a constraint solver with the python‑constraint package.

Please output the JSON object following this exact schema:

{
    "variables": [
        {
            "name": "<variable_name_as_string>",
            "domain": [ <list_of_possible_values> ]
        },
        ...
    ],
    "constraints": [
        {
            "variables": [ "<var1>", "<var2>", ... ],
            "expression": "<Python expression string representing the constraint>"
        },
        ...
    ]
}

Guidelines:
1. **Variables:**
   - Identify all variables mentioned (typically represented by letters or names).
   - For each variable, if the domain (the list of possible values) is explicitly defined in the puzzle, use that; otherwise, infer a default domain based on context.
     For example, if the puzzle involves countries and mentions "United Kingdom" and "United States", use the domain `["UK", "US"]`.
2. **Constraints:**
   - Convert each condition statement into a constraint expression.
   - Use a Python logical expression format that can be passed to the python‑constraint package.
   - Reference variables exactly as they appear.
   - For example, given the statement:
     "If G goes to the UK, then H goes to the United States."
     a possible constraint expression is:
     `"G != 'UK' or H == 'US'"`
     and the corresponding constraint object should include the variable names `["G", "H"]`.
3. **Output:**
   - Do not include any additional text or explanation; output only the JSON object.

For example, if the puzzle description is:

"There are 7 outstanding students G, H, L, M, U, W and Z in a school. During the summer vacation, the school will send them to the United Kingdom and the United States for inspection. The school has only 7 students participating in this activity, and each person happens to go to one of these two countries. Considering the specialty of each student, the activity must meet the following conditions:
(1) If G goes to the UK, then H goes to the United States.
(2) If L goes to the UK, then both M and U go to the US.
(3) The country W went to is different from the country Z went to.
(4) The country where U goes is different from the country where G goes.
(5) If Z goes to the UK, then H also goes to the UK."

Your output should include:
- Variables: G, H, L, M, U, W, Z (each with an appropriate domain, e.g., `["UK", "US"]`)
- Constraints:
   - For condition (1): `{"variables": ["G", "H"], "expression": "G != 'UK' or H == 'US'"}`
   - For condition (2): `{"variables": ["L", "M", "U"], "expression": "L != 'UK' or (M == 'US' and U == 'US')"}`
   - For condition (3): `{"variables": ["W", "Z"], "expression": "W != Z"}`
   - For condition (4): `{"variables": ["U", "G"], "expression": "U != G"}`
   - For condition (5): `{"variables": ["Z", "H"], "expression": "Z != 'UK' or H == 'UK'"}`

Now, please process the following puzzle description and output the corresponding JSON:

"There's the puzzle description here:
'There are 7 outstanding students G, H, L, M, U, W and Z in a school. During the summer vacation, the school will send them to the United Kingdom and the United States for inspection. The school has only 7 students participating in this activity, and each person happens to go to one of these two countries. Considering the specialty of each student, the activity must meet the following conditions: (1) If G goes to the UK, then H goes to the United States. (2) If L goes to the UK, then both M and U go to the US. (3) The country W went to is different from the country Z went to. (4) The country where U goes is different from the country where G goes. (5) If Z goes to the UK, then H also goes to the UK.'"

Output only the JSON.
"""

solver_prompt = "Here are the statements: " # + str(question_parsing)

In [72]:
solver_structure = llm.call_model(system_prompt=solver_system_prompt, prompt=solver_prompt, deployment_name=deployment_name)

In [73]:
solver_structure

'{\n    "variables": [\n        {\n            "name": "G_position",\n            "domain": ["front_1", "front_2", "middle_1", "middle_2", "back_1", "back_2", "not_playing"]\n        },\n        {\n            "name": "H_position",\n            "domain": ["front_1", "front_2", "middle_1", "middle_2", "back_1", "back_2", "not_playing"]\n        },\n        {\n            "name": "K_position",\n            "domain": ["front_1", "front_2", "middle_1", "middle_2", "back_1", "back_2", "not_playing"]\n        },\n        {\n            "name": "L_position",\n            "domain": ["front_1", "front_2", "middle_1", "middle_2", "back_1", "back_2", "not_playing"]\n        },\n        {\n            "name": "N_position",\n            "domain": ["front_1", "front_2", "middle_1", "middle_2", "back_1", "back_2", "not_playing"]\n        },\n        {\n            "name": "P_position",\n            "domain": ["front_1", "front_2", "middle_1", "middle_2", "back_1", "back_2", "not_playing"]\n        },

In [74]:
json.loads(solver_structure)

{'variables': [{'name': 'G_position',
   'domain': ['front_1',
    'front_2',
    'middle_1',
    'middle_2',
    'back_1',
    'back_2',
    'not_playing']},
  {'name': 'H_position',
   'domain': ['front_1',
    'front_2',
    'middle_1',
    'middle_2',
    'back_1',
    'back_2',
    'not_playing']},
  {'name': 'K_position',
   'domain': ['front_1',
    'front_2',
    'middle_1',
    'middle_2',
    'back_1',
    'back_2',
    'not_playing']},
  {'name': 'L_position',
   'domain': ['front_1',
    'front_2',
    'middle_1',
    'middle_2',
    'back_1',
    'back_2',
    'not_playing']},
  {'name': 'N_position',
   'domain': ['front_1',
    'front_2',
    'middle_1',
    'middle_2',
    'back_1',
    'back_2',
    'not_playing']},
  {'name': 'P_position',
   'domain': ['front_1',
    'front_2',
    'middle_1',
    'middle_2',
    'back_1',
    'back_2',
    'not_playing']},
  {'name': 'Q_position',
   'domain': ['front_1',
    'front_2',
    'middle_1',
    'middle_2',
    'back_1',


In [75]:
import json
from constraint import Problem

def build_solver_from_json(config):
    problem = Problem()

    # Get the set of defined variable names.
    defined_vars = {var["name"] for var in config["variables"]}

    # Verify that all variables referenced in constraints are defined.
    for constr in config["constraints"]:
        for var in constr["variables"]:
            if var not in defined_vars:
                raise ValueError(f"Constraint references variable '{var}' which is not defined in the variables list.")

    # Add all defined variables to the problem.
    for var in config["variables"]:
        name = var["name"]
        domain = var["domain"]
        problem.addVariable(name, domain)

    # Add constraints.
    for constr in config["constraints"]:
        var_list = constr["variables"]
        expression = constr["expression"]
        # Create a lambda function string.
        args = ", ".join(var_list)
        lambda_str = f"lambda {args}: {expression}"
        try:
            constraint_func = eval(lambda_str)
        except Exception as e:
            raise ValueError(f"Error evaluating lambda expression: {lambda_str}") from e
        problem.addConstraint(constraint_func, var_list)

    return problem

In [76]:
solver = build_solver_from_json(config=json.loads(solver_structure))
solutions = solver.getSolutions()
print("Solutions found:")
print(solutions)

Solutions found:
[{'N_position': 'not_playing', 'P_position': 'not_playing', 'Q_position': 'not_playing', 'H_position': 'not_playing', 'K_position': 'not_playing', 'G_position': 'not_playing', 'L_position': 'not_playing'}, {'N_position': 'not_playing', 'P_position': 'not_playing', 'Q_position': 'not_playing', 'H_position': 'not_playing', 'K_position': 'not_playing', 'G_position': 'not_playing', 'L_position': 'back_1'}, {'N_position': 'not_playing', 'P_position': 'not_playing', 'Q_position': 'not_playing', 'H_position': 'not_playing', 'K_position': 'not_playing', 'G_position': 'not_playing', 'L_position': 'middle_1'}, {'N_position': 'not_playing', 'P_position': 'not_playing', 'Q_position': 'not_playing', 'H_position': 'not_playing', 'K_position': 'not_playing', 'G_position': 'not_playing', 'L_position': 'front_1'}, {'N_position': 'not_playing', 'P_position': 'not_playing', 'Q_position': 'not_playing', 'H_position': 'not_playing', 'K_position': 'not_playing', 'G_position': 'front_2', 'L_